# Computational Theory Assessment

## Problem 1: Binary Words and Operations

In [26]:
import numpy as np

def Parity(x, y, z):
    """
    SHA-1 Parity function: x ^ y ^ z.
    
    Computes the bitwise XOR of three 32-bit words.
    This function is used in SHA-1 and returns 1 for each bit position
    where an odd number of the corresponding bits in x, y, z are 1.
    
    Args:
        x: 32-bit unsigned integer
        y: 32-bit unsigned integer
        z: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of x XOR y XOR z
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    result = x ^ y ^ z

    return np.uint32(result)

In [27]:
def test_parity():
    """Test the Parity function with various inputs."""
    
    # Test 1: Identity - all zeros
    assert Parity(0x00000000, 0x00000000, 0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: Single bit set
    assert Parity(0x00000001, 0x00000000, 0x00000000) == 0x00000001, \
        "Single 1 bit should return 1"
    
    # Test 3: Two bits set (even parity)
    assert Parity(0x00000001, 0x00000001, 0x00000000) == 0x00000000, \
        "Two 1 bits should return 0 (even parity cancels)"
    
    # Test 4: Three bits set (odd parity)
    assert Parity(0x00000001, 0x00000001, 0x00000001) == 0x00000001, \
        "Three 1 bits should return 1 (odd parity)"
    
    # Test 5: All ones
    assert Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 6: Complementary patterns
    assert Parity(0xAAAAAAAA, 0x55555555, 0xFFFFFFFF) == 0x00000000, \
        "Complementary patterns should cancel to zero"
    
    # Test 7: Mixed bits test
    assert Parity(0x00000005, 0x00000003, 0x00000001) == 0x00000007, \
        "Mixed bits: 0101 XOR 0011 XOR 0001 = 0111"
    
    # Test 8: 4-bit test
    assert Parity(0b1010, 0b1100, 0b0011) == 0b0101, \
        "Binary: 1010 XOR 1100 XOR 0011 = 0101"
    
    # Test 9: SHA-1 constants
    expected = np.uint32(0x67452301 ^ 0xEFCDAB89 ^ 0x98BADCFE)
    assert Parity(0x67452301, 0xEFCDAB89, 0x98BADCFE) == expected, \
        "SHA-1 initial hash values test"
    
    # Test 10: Commutativity
    assert Parity(0x12345678, 0xABCDEF01, 0xFEDCBA98) == \
           Parity(0xFEDCBA98, 0xABCDEF01, 0x12345678), \
        "XOR is commutative - order shouldn't matter"
    
    print("✓ All Parity tests passed")

# Run the tests
test_parity()

✓ All Parity tests passed


In [28]:
def Ch(x, y, z):
    """
    SHA-1 Ch (Choose) function: (x & y) ^ (~x & z).
    
    The Ch function "chooses" between y and z based on the bits of x:
    - If a bit in x is 1, the corresponding bit from y is chosen
    - If a bit in x is 0, the corresponding bit from z is chosen
    
    Args:
        x: 32-bit unsigned integer (selector)
        y: 32-bit unsigned integer (chosen when x bit is 1)
        z: 32-bit unsigned integer (chosen when x bit is 0)
    
    Returns:
        32-bit unsigned integer result of (x & y) ^ (~x & z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # (x AND y) XOR (NOT x AND z)
    result = (x & y) ^ (~x & z)
    
    return np.uint32(result)

In [29]:
# Test the Ch function
print("Testing SHA-1 Ch (Choose) Function")
print("=" * 60)

# Test 1: All ones in x (choose all from y)
print("Test 1: All ones in x")
result1 = Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF00)
print(f"  Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF00)")
print(f"  Result:   0x{result1:08x}")
print(f"  Expected: 0x12345678")
print(f"  Pass: {result1 == 0x12345678}\n")

# Test 2: All zeros in x (choose all from z)
print("Test 2: All zeros in x")
result2 = Ch(0x00000000, 0x12345678, 0xABCDEF00)
print(f"  Ch(0x00000000, 0x12345678, 0xABCDEF00)")
print(f"  Result:   0x{result2:08x}")
print(f"  Expected: 0xABCDEF00")
print(f"  Pass: {result2 == 0xABCDEF00}\n")

# Test 3: Alternating pattern
print("Test 3: Alternating pattern")
result3 = Ch(0xAAAAAAAA, 0xFFFFFFFF, 0x00000000)
print(f"  Ch(0xAAAAAAAA, 0xFFFFFFFF, 0x00000000)")
print(f"  Binary breakdown:")
print(f"    x = 10101010 (choose y where 1, z where 0)")
print(f"    y = 11111111")
print(f"    z = 00000000")
print(f"  Result:   0x{result3:08x}")
print(f"  Expected: 0xAAAAAAAA")
print(f"  Pass: {result3 == 0xAAAAAAAA}\n")

# Test 4: Binary example
print("Test 4: Binary")
result4 = Ch(0b1100, 0b1010, 0b0101)
print(f"  Ch(0b1100, 0b1010, 0b0101)")
print(f"  Binary breakdown:")
print(f"    x = 00000000000000000000000000001100 (selector)")
print(f"    y = 00000000000000000000000000001010 (choose when x=1)")
print(f"    z = 00000000000000000000000000000101 (choose when x=0)")
print(f"    Result: bits 3,2=1 choose y, bits 1,0=0 choose z")
print(f"    Expected = 00000000000000000000000000001001 (9)")
print(f"  Result:   0b{int(result4):04b} = {result4}")
print(f"  Expected: 9")
print(f"  Pass: {result4 == 9}\n")

# Test 5: Actual hash values
print("Test 5: Actual hash values")
result5 = Ch(0x67452301, 0xEFCDAB89, 0x98BADCFE)
expected5 = np.uint32((0x67452301 & 0xEFCDAB89) ^ (~np.uint32(0x67452301) & 0x98BADCFE))
print(f"  Ch(0x67452301, 0xEFCDAB89, 0x98BADCFE)")
print(f"  Result:   0x{result5:08x}")
print(f"  Expected: 0x{expected5:08x}")
print(f"  Pass: {result5 == expected5}")

print("=" * 60)

Testing SHA-1 Ch (Choose) Function
Test 1: All ones in x
  Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF00)
  Result:   0x12345678
  Expected: 0x12345678
  Pass: True

Test 2: All zeros in x
  Ch(0x00000000, 0x12345678, 0xABCDEF00)
  Result:   0xabcdef00
  Expected: 0xABCDEF00
  Pass: True

Test 3: Alternating pattern
  Ch(0xAAAAAAAA, 0xFFFFFFFF, 0x00000000)
  Binary breakdown:
    x = 10101010 (choose y where 1, z where 0)
    y = 11111111
    z = 00000000
  Result:   0xaaaaaaaa
  Expected: 0xAAAAAAAA
  Pass: True

Test 4: Binary
  Ch(0b1100, 0b1010, 0b0101)
  Binary breakdown:
    x = 00000000000000000000000000001100 (selector)
    y = 00000000000000000000000000001010 (choose when x=1)
    z = 00000000000000000000000000000101 (choose when x=0)
    Result: bits 3,2=1 choose y, bits 1,0=0 choose z
    Expected = 00000000000000000000000000001001 (9)
  Result:   0b1001 = 9
  Expected: 9
  Pass: True

Test 5: Actual hash values
  Ch(0x67452301, 0xEFCDAB89, 0x98BADCFE)
  Result:   0xffffffff
  Expec

In [30]:
def Maj(x, y, z):
    """
    SHA-1 Maj (Majority) function: (x & y) ^ (x & z) ^ (y & z).
    
    The Maj function returns the majority vote for each bit position:
    - Returns 1 if at least 2 of the 3 corresponding bits are 1
    - Returns 0 if at least 2 of the 3 corresponding bits are 0
    
    Args:
        x: 32-bit unsigned integer
        y: 32-bit unsigned integer
        z: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of (x & y) ^ (x & z) ^ (y & z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # (x AND y) XOR (x AND z) XOR (y AND z)
    result = (x & y) ^ (x & z) ^ (y & z)
    
    return np.uint32(result)

In [31]:
# Test the Maj function
print("Testing SHA-1 Maj (Majority) Function")
print("=" * 60)

# Test 1: All zeros
print("Test 1: All zeros")
result1 = Maj(0x00000000, 0x00000000, 0x00000000)
print(f"  Maj(0x00000000, 0x00000000, 0x00000000)")
print(f"  Result:   0x{result1:08x}")
print(f"  Expected: 0x00000000")
print(f"  Pass: {result1 == 0x00000000}\n")

# Test 2: All ones
print("Test 2: All ones")
result2 = Maj(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF)
print(f"  Maj(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF)")
print(f"  Result:   0x{result2:08x}")
print(f"  Expected: 0xFFFFFFFF")
print(f"  Pass: {result2 == 0xFFFFFFFF}\n")

# Test 3: Two ones, one zero (majority = 1)
print("Test 3: Two ones, one zero")
result3 = Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000)
print(f"  Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000)")
print(f"  Binary breakdown:")
print(f"    x = 11111111... (all ones)")
print(f"    y = 11111111... (all ones)")
print(f"    z = 00000000... (all zeros)")
print(f"    Majority at each bit: 2 ones, 1 zero → 1")
print(f"  Result:   0x{result3:08x}")
print(f"  Expected: 0xFFFFFFFF")
print(f"  Pass: {result3 == 0xFFFFFFFF}\n")

# Test 4: Binary example
print("Test 4: Binary")
result4 = Maj(0b1100, 0b1010, 0b1001)
print(f"  Maj(0b1100, 0b1010, 0b1001)")
print(f"  Binary breakdown:")
print(f"    Position: 3  2  1  0")
print(f"    x:        1  1  0  0")
print(f"    y:        1  0  1  0")
print(f"    z:        1  0  0  1")
print(f"    Bit 0: 0,0,1 → majority is 0")
print(f"    Bit 1: 0,1,0 → majority is 0")
print(f"    Bit 2: 1,0,0 → majority is 0")
print(f"    Bit 3: 1,1,1 → majority is 1")
print(f"    Result = 1000 (binary) = 8")
print(f"  Result:   0b{int(result4):04b} = {result4}")
print(f"  Expected: 8")
print(f"  Pass: {result4 == 8}\n")

# Test 5: Actual hash values
print("Test 5: Actual hash values")
result5 = Maj(0x67452301, 0xEFCDAB89, 0x98BADCFE)
expected5 = np.uint32((0x67452301 & 0xEFCDAB89) ^ (0x67452301 & 0x98BADCFE) ^ (0xEFCDAB89 & 0x98BADCFE))
print(f"  Maj(0x67452301, 0xEFCDAB89, 0x98BADCFE)")
print(f"  Result:   0x{result5:08x}")
print(f"  Expected: 0x{expected5:08x}")
print(f"  Pass: {result5 == expected5}")

print("=" * 60)

Testing SHA-1 Maj (Majority) Function
Test 1: All zeros
  Maj(0x00000000, 0x00000000, 0x00000000)
  Result:   0x00000000
  Expected: 0x00000000
  Pass: True

Test 2: All ones
  Maj(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF)
  Result:   0xffffffff
  Expected: 0xFFFFFFFF
  Pass: True

Test 3: Two ones, one zero
  Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000)
  Binary breakdown:
    x = 11111111... (all ones)
    y = 11111111... (all ones)
    z = 00000000... (all zeros)
    Majority at each bit: 2 ones, 1 zero → 1
  Result:   0xffffffff
  Expected: 0xFFFFFFFF
  Pass: True

Test 4: Binary
  Maj(0b1100, 0b1010, 0b1001)
  Binary breakdown:
    Position: 3  2  1  0
    x:        1  1  0  0
    y:        1  0  1  0
    z:        1  0  0  1
    Bit 0: 0,0,1 → majority is 0
    Bit 1: 0,1,0 → majority is 0
    Bit 2: 1,0,0 → majority is 0
    Bit 3: 1,1,1 → majority is 1
    Result = 1000 (binary) = 8
  Result:   0b1000 = 8
  Expected: 8
  Pass: True

Test 5: Actual hash values
  Maj(0x67452301, 0xEFCDAB89,

In [32]:
def Sigma0(x):
    """
    SHA-256 Sigma0 (Σ₀²⁵⁶) function: ROTR²(x) ^ ROTR¹³(x) ^ ROTR²²(x)
    
    This function performs three right rotations on a 32-bit word and XORs them.
    
    ROTR^n(x) = circular right rotation of x by n positions
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR²(x) ^ ROTR¹³(x) ^ ROTR²²(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 2, 13, and 22 bits
    rotr2 = np.uint32((x >> 2) | (x << 30))   # ROTR²(x)
    rotr13 = np.uint32((x >> 13) | (x << 19)) # ROTR¹³(x)
    rotr22 = np.uint32((x >> 22) | (x << 10)) # ROTR²²(x)
    
    # XOR all three rotations
    result = rotr2 ^ rotr13 ^ rotr22
    
    return np.uint32(result)

In [33]:
# Test the Sigma0 function
print("Testing SHA-256 Sigma0 (Σ₀²⁵⁶) Function")
print("=" * 60)

# Test 1: All zeros
print("Test 1: All zeros")
result1 = Sigma0(0x00000000)
print(f"  Σ₀²⁵⁶(0x00000000)")
print(f"  Result:   0x{result1:08x}")
print(f"  Expected: 0x00000000")
print(f"  Pass: {result1 == 0x00000000}\n")

# Test 2: All ones
print("Test 2: All ones")
result2 = Sigma0(0xFFFFFFFF)
print(f"  Σ₀²⁵⁶(0xFFFFFFFF)")
print(f"  Result:   0x{result2:08x}")
print(f"  Expected: 0xFFFFFFFF (all rotations of all 1s = all 1s)")
print(f"  Pass: {result2 == 0xFFFFFFFF}\n")


print("=" * 60)

Testing SHA-256 Sigma0 (Σ₀²⁵⁶) Function
Test 1: All zeros
  Σ₀²⁵⁶(0x00000000)
  Result:   0x00000000
  Expected: 0x00000000
  Pass: True

Test 2: All ones
  Σ₀²⁵⁶(0xFFFFFFFF)
  Result:   0xffffffff
  Expected: 0xFFFFFFFF (all rotations of all 1s = all 1s)
  Pass: True



In [34]:
def Sigma1(x):
    """
    SHA-256 Sigma1 (Σ₁²⁵⁶) function: ROTR⁶(x) ^ ROTR¹¹(x) ^ ROTR²⁵(x)
    
    This function performs three right rotations on a 32-bit word and XORs them.
    
    ROTR^n(x) = circular right rotation of x by n positions
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR⁶(x) ^ ROTR¹¹(x) ^ ROTR²⁵(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 6, 11, and 25 bits
    rotr6 = np.uint32((x >> 6) | (x << 26))   # ROTR⁶(x)
    rotr11 = np.uint32((x >> 11) | (x << 21)) # ROTR¹¹(x)
    rotr25 = np.uint32((x >> 25) | (x << 7))  # ROTR²⁵(x)
    
    # XOR all three rotations
    result = rotr6 ^ rotr11 ^ rotr25
    
    return np.uint32(result)

In [35]:
# Test the Sigma1 function
print("Testing SHA-256 Sigma1 (Σ₁²⁵⁶) Function")
print("=" * 60)

# Test 1: All zeros
print("Test 1: All zeros")
result1 = Sigma1(0x00000000)
print(f"  Σ₁²⁵⁶(0x00000000)")
print(f"  Result:   0x{result1:08x}")
print(f"  Expected: 0x00000000")
print(f"  Pass: {result1 == 0x00000000}\n")

# Test 2: All ones
print("Test 2: All ones")
result2 = Sigma1(0xFFFFFFFF)
print(f"  Σ₁²⁵⁶(0xFFFFFFFF)")
print(f"  Result:   0x{result2:08x}")
print(f"  Expected: 0xFFFFFFFF (all rotations of all 1s = all 1s)")
print(f"  Pass: {result2 == 0xFFFFFFFF}\n")

print("=" * 60)

Testing SHA-256 Sigma1 (Σ₁²⁵⁶) Function
Test 1: All zeros
  Σ₁²⁵⁶(0x00000000)
  Result:   0x00000000
  Expected: 0x00000000
  Pass: True

Test 2: All ones
  Σ₁²⁵⁶(0xFFFFFFFF)
  Result:   0xffffffff
  Expected: 0xFFFFFFFF (all rotations of all 1s = all 1s)
  Pass: True



In [36]:
def sigma0(x):
    """
    SHA-256 sigma0 (σ₀²⁵⁶) function: ROTR⁷(x) ^ ROTR¹⁸(x) ^ SHR³(x)
    
    This function performs two right rotations and one right shift on a 32-bit word and XORs them.
    Used in the message schedule of SHA-256.
    
    ROTR^n(x) = circular right rotation of x by n positions
    SHR^n(x) = right shift of x by n positions (no wrap around)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR⁷(x) ^ ROTR¹⁸(x) ^ SHR³(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 7 and 18 bits
    rotr7 = np.uint32((x >> 7) | (x << 25))   # ROTR⁷(x)
    rotr18 = np.uint32((x >> 18) | (x << 14)) # ROTR¹⁸(x)
    
    # Right shift by 3 bits (no wrap around)
    shr3 = np.uint32(x >> 3)                  # SHR³(x)
    
    # XOR all three operations
    result = rotr7 ^ rotr18 ^ shr3
    
    return np.uint32(result)

In [37]:
def sigma1(x):
    """
    SHA-256 sigma1 (σ₁²⁵⁶) function: ROTR¹⁷(x) ^ ROTR¹⁹(x) ^ SHR¹⁰(x)
    
    This function performs two right rotations and one right shift on a 32-bit word and XORs them.
    Used in the message schedule of SHA-256 (extending the message).
    
    ROTR^n(x) = circular right rotation of x by n positions
    SHR^n(x) = right shift of x by n positions (no wrap around, fills with zeros)
    
    Reference: FIPS 180-4, Section 4.1.2 (SHA-256 Functions)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR¹⁷(x) ^ ROTR¹⁹(x) ^ SHR¹⁰(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 17 and 19 bits
    rotr17 = np.uint32((x >> 17) | (x << 15))  # ROTR¹⁷(x)
    rotr19 = np.uint32((x >> 19) | (x << 13))  # ROTR¹⁹(x)
    
    # Right shift by 10 bits (no wrap around)
    shr10 = np.uint32(x >> 10)                 # SHR¹⁰(x)
    
    # XOR all three operations
    result = rotr17 ^ rotr19 ^ shr10
    
    return np.uint32(result)

## Problem 2: Fractional Parts of Cube Roots


## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords

# End